### Publish Model API

In [1]:
#set up API
import requests
import json
import os

#set up API key and headers
api_key = os.environ['DOMINO_USER_API_KEY']
headers = {'X-Domino-Api-Key': api_key,  'Content-Type': 'application/json'}  

#api host
host = os.environ['DOMINO_API_HOST']

In [2]:
os.environ['DOMINO_USER_API_KEY'] = "e96911141303c50867a8f331b4abdf0ca9f7719e7111b605d87fcec7d6b5ca29"
os.environ['DOMINO_API_HOST'] = "https://cdc-sandbox.domino-eval.com"

host

'http://nucleus-frontend.domino-platform:80'

In [3]:
import os


    
from domino import Domino

domino = Domino(
    "pok8/Heart-Attack-prediction",
    api_key="e96911141303c50867a8f331b4abdf0ca9f7719e7111b605d87fcec7d6b5ca29",
    host="https://cdc-sandbox.domino-eval.com"
)



In [4]:
#get user id
url_user = '{host}/v4/users/self'.format(host=host)
r_user = requests.get(url_user, headers=headers)
user_id = r_user.json()['id']

#get project id
project_name = os.environ['DOMINO_PROJECT_NAME']
url_project = '{host}/v4/projects?name={project_name}&ownerId={user_id}'.format(
    host=host, project_name=project_name, user_id=user_id)
r_project = requests.get(url_project, headers=headers)
projectId = r_project.json()[0]['id']


In [5]:
#get default environment ID for the project
env_url = '{host}/v4/projects/{projectId}/settings'.format(host=host, projectId=projectId)
r_env = requests.get(env_url, headers=headers)
env_id = r_env.json()['defaultEnvironmentId']
env_id

'650255d38f2b224c6b1e2ab9'

In [6]:
#publish the model endpoint

## make sure to fill out below info ##
filename = './model_src/heart_attack_model.py'
function = 'predict'
model_name = 'Auto model'
model_desc = 'Created using auto deploy'
commitId =  "ae5eb1bc5d5df130b9e70c1cce9d3930a9eb262d"
publish_url = '{host}/v1/models'.format(host=host)

details = {
  "projectId": projectId,
  "file": filename,
  "function": function,
  "environmentId": env_id,
  "name": model_name,
  "description": model_desc
}

r = requests.post(publish_url, headers=headers, json=details)
r.json()

{'data': {'_id': '6531a60b62be6659a6f8aacc',
  'name': 'Auto model',
  'description': 'Created using auto deploy',
  'isArchived': False,
  'replicas': 2,
  'metadata': {'createdBy': '650465090a7f750eb5f7335a',
   'created': 1697752587285,
   'lastModified': 1697752587285},
  'collaboratorSettings': {'UserRef(650465090a7f750eb5f7335a)': 'Owner',
   'EveryoneRef': 'Hidden'},
  'accessConfiguration': {'isPublic': False,
   'accessTokens': [{'_id': '6531a60b62be6659a6f8aacb',
     'created': 1697752587274,
     'createdBy': '650465090a7f750eb5f7335a',
     'lastGenerated': 1697752587274,
     'lastGeneratedBy': '650465090a7f750eb5f7335a'}]}}}

## Export Model API to External Registry

In [ ]:
#list model API in the current project
project_name = os.environ['DOMINO_PROJECT_NAME']
username = os.environ['DOMINO_PROJECT_OWNER']
modelapis_url = '{host}/v1/projects/{username}/{project_name}/models'.format(host=host, username=username,
                                                                         project_name=project_name)
modelapis_request = requests.get(modelapiss_url, headers=headers)
modelapis = modelapis_request.json()['data']
print(modelapis)
modelapis_id = modelapis[0]['id']
print(modelapis_id)

In [ ]:
#list versions of a chosen model API
versions_url = '{host}/v1/models/{model_id}/versions'.format(host=host, model_id=model_id)
versions_request = requests.get(versions_url, headers=headers)
versions = []

for x in range(len(versions_request.json()['data'])):
    versions.append(versions_request.json()['data'][x]['_id'])

versions

In [ ]:
#set your model version ID - defaults to the first one
modelVersionId = versions[0]
modelVersionId

In [ ]:
#export model
export_url = '{host}/v4/models/{modelId}/{modelVersionId}/exportImageToRegistry'.format(host=host,
                                                            modelId=model_id, modelVersionId=modelVersionId)

#set export parameters
#examples
#      "registryUrl": "quay.io",
#      "repository": "andrea_lowe/exported_model",
#      "password": "<your token/password>",
#      "tag": "test",
#      "username": "andrea_lowe"

repo_password = os.environ['<password that you stored as an environment variable>']
tag = '<tag>'

values = {"registryUrl": "<url>", "repository": "<username/reponame>", "password": repo_password,
          "tag": tag, "username": "<username>"}

r = requests.post(export_url, headers=headers, json=values)
r.json()

In [ ]:
exportId = r.json()['exportId']
exportId

In [ ]:
#export status
status_url = '{host}/v4/models/{exportId}/getExportImageStatus'.format(host=host,exportId=exportId)
status_request = requests.get(status_url, headers=headers)
status = status_request.json()
status

## API example for using Model Registry

In [14]:
#list all registered models visible to you

models_url = '{host}/api/registeredmodels/beta/names'.format(host=host)
models_request = requests.get(models_url, headers=headers)
models = models_request.json()
models

{'items': ['Andrea-Test',
  'custom_heart_attack_model_api',
  'demo-1',
  'demo-2',
  'demo-test-01',
  'direct_heart_attack_model',
  'heart-attack-LR',
  'heart-attack-LR-model',
  'heart-attack-model-v2',
  'heart-attack-prediction-1',
  'heart-attack-random-forest-v1',
  'heart_attack_sklearn_RF',
  'lgb_heart_attack_model',
  'original-sklearn_model_api',
  'sk_learn_model_RF_1',
  'sklearn_RF_model',
  'SVM-digits',
  'Test'],
 'metadata': {'requestId': 'aab16e64-b0e4-4ded-9daf-d682278d7a58',
  'notices': [],
  'offset': 0,
  'limit': 0,
  'totalCount': 1}}